In [44]:
import pandas as pd
import numpy as np
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import itertools
import mediapipe as mp
from collections import deque
import matplotlib.pyplot  as plt
import torch

In [4]:
filtered_df = pd.read_csv('Mediapipe_Dataset_with_0.csv')

In [5]:
for i in range(21):
    filtered_df.drop(columns = f'landmark_{i}_z', inplace = True)

In [6]:
X, y = filtered_df.drop(columns=['label','label_id', 'path']), filtered_df['label_id']

In [7]:
minmax = RobustScaler()
X_scaled = minmax.fit_transform(X)

window_size_x = 37
window_size_y = 37
print(len(X), len(y))
n_samples = len(X_scaled) // window_size_x
n_samples_y = len(y) // window_size_y

y_seq = np.array(y[:n_samples_y * window_size_y]).reshape(n_samples_y, window_size_y)

y_seq = y_seq[:, 0] 

print(y_seq.shape)

X_scaled = X_scaled.reshape(n_samples, 37, 42)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_seq, train_size=0.8, random_state=13)
print(X_scaled.shape)

492174 492174
(13302,)
(13302, 37, 42)


In [21]:
X_tensor_train = torch.tensor(X_train).float()
X_tensor_test = torch.tensor(X_test).float()

In [12]:
import torch
import torch.nn as nn

class GRUAutoencoder(nn.Module):
    def __init__(self, input_dim=63, hidden_dim=128, latent_dim=32):
        super(GRUAutoencoder, self).__init__()

        self.encoder_gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.latent = nn.Linear(hidden_dim, latent_dim) 

        self.decoder_init = nn.Linear(latent_dim, hidden_dim)
        self.decoder_gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):

        _, h_n = self.encoder_gru(x) 
        h_n = h_n.squeeze(0)
        z = self.latent(h_n)


        h0_dec = self.decoder_init(z).unsqueeze(0)
        decoder_input = torch.zeros_like(x)  

        dec_out, _ = self.decoder_gru(decoder_input, h0_dec)
        x_reconstructed = self.output_layer(dec_out)

        return x_reconstructed, z


In [13]:
def train_autoencoder(model, X_tensor, epochs=50, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        x_recon, z = model(X_tensor)   
        loss = loss_fn(x_recon, X_tensor)
        loss.backward()                  
        optimizer.step()                  
        print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")


In [ ]:
model = GRUAutoencoder(input_dim=42, hidden_dim=128, latent_dim=32)

train_autoencoder(model, X_tensor_train, epochs=50, lr=1e-3)

Epoch 1: Loss = 0.8235
Epoch 2: Loss = 0.8120
Epoch 3: Loss = 0.8006
Epoch 4: Loss = 0.7891
Epoch 5: Loss = 0.7773
Epoch 6: Loss = 0.7648
Epoch 7: Loss = 0.7515
Epoch 8: Loss = 0.7370
Epoch 9: Loss = 0.7211
Epoch 10: Loss = 0.7034
Epoch 11: Loss = 0.6840
Epoch 12: Loss = 0.6622
Epoch 13: Loss = 0.6376
Epoch 14: Loss = 0.6109
Epoch 15: Loss = 0.5880
Epoch 16: Loss = 0.5896
Epoch 17: Loss = 0.6025
Epoch 18: Loss = 0.5909
Epoch 19: Loss = 0.5767
Epoch 20: Loss = 0.5699
Epoch 21: Loss = 0.5688
Epoch 22: Loss = 0.5696
Epoch 23: Loss = 0.5700
Epoch 24: Loss = 0.5690
Epoch 25: Loss = 0.5665
Epoch 26: Loss = 0.5628
Epoch 27: Loss = 0.5587
Epoch 28: Loss = 0.5551
Epoch 29: Loss = 0.5523
Epoch 30: Loss = 0.5493
Epoch 31: Loss = 0.5435
Epoch 32: Loss = 0.5342
Epoch 33: Loss = 0.5229
Epoch 34: Loss = 0.5106
Epoch 35: Loss = 0.4955
Epoch 36: Loss = 0.4760
Epoch 37: Loss = 0.4545
Epoch 38: Loss = 0.4389
Epoch 39: Loss = 0.4231
Epoch 40: Loss = 0.4061
Epoch 41: Loss = 0.3995
Epoch 42: Loss = 0.3886
E

In [ ]:
model.eval()

with torch.no_grad():
    x_reconstructed, z_test = model(X_tensor_test)

    loss_fn = nn.MSELoss()
    test_loss = loss_fn(x_reconstructed, X_tensor_test)

print(f"Test reconstruction loss (MSE): {test_loss.item():.4f}")

Test reconstruction loss (MSE): 0.3578


In [24]:
model.eval() 
with torch.no_grad():
    _, z_train = model(X_tensor_train)
    _, z_test = model(X_tensor_test)

In [ ]:
z_train_np = z_train.cpu().numpy()
z_test_np = z_test.cpu().numpy()

In [42]:
robust = RobustScaler()
z_train_sc = robust.fit_transform(z_train_np)
z_test_sc = robust.transform(z_test_np)

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression(max_iter=1000)
clf.fit(z_train_sc, y_train)

y_pred = clf.predict(z_test_sc)
acc = accuracy_score(y_test, y_pred)

print(f"Classification accuracy: {acc:.4f}")

Classification accuracy: 0.4927
